## Analytical Data Mining: Homework 5
**Name**: Advait Ramesh Iyer

**NetID**: aiyer01 **|** **SUID:** 330623485

### Question 3

Flajolet Martin Algorithm:

**Step 1:** Imported all the necessary packages, and retrieved file list.

In [3]:
import glob
import pyhash
import sklearn
import numpy as np
import csv
from sklearn.utils.murmurhash import murmurhash3_32
files = glob.glob("./*.txt")
files

['./quotes_2009-01.txt',
 './quotes_2009-02.txt',
 './quotes_2009-03.txt',
 './quotes_2009-04.txt',
 './quotes_2008-09.txt',
 './quotes_2008-08.txt',
 './quotes_2008-11.txt',
 './quotes_2008-10.txt',
 './quotes_2008-12.txt']

**Step 2:** Built hasher function, which filters only quotes. It then calculates the trailing zeros in the output of Murmur Hash on each quote.

In [9]:
def Hasher(files):
    for f in files:
        with open(f) as fval:
            while(True):
                line = fval.readline()
                if not line:
                    break
                if len(line) == 0:
                    continue
                if line[0] == "Q":
                    temp = []
                    for i in range(40):
                        murmur = murmurhash3_32(line, positive = True, seed = i)
                        bin_murmur = bin(murmur)
                        trail_murmur = len(bin_murmur[2:]) - (bin_murmur[2:].rfind("1") + 1)
                        temp.append(trail_murmur)
                    yield temp
                    
Rs = Hasher(files)
Rs

<generator object Hasher at 0xa1f1d1a50>

**Step 3:** Processed chunks of 200,000 lines, and calculated the max number of trailing zeros in each chunk.

In [10]:
max_zero = [-1 for i in range(40)]
ctr = 0
for R in Rs:
    if ctr % 200000 == 0:
        print("{} lines processed".format(ctr)) 
    ctr += 1
    for j in range(40):
        if R[j] > max_zero[j]:
            max_zero[j] = R[j]

0 lines processed
200000 lines processed
400000 lines processed
600000 lines processed
800000 lines processed
1000000 lines processed
1200000 lines processed
1400000 lines processed
1600000 lines processed
1800000 lines processed
2000000 lines processed
2200000 lines processed
2400000 lines processed
2600000 lines processed
2800000 lines processed
3000000 lines processed
3200000 lines processed
3400000 lines processed
3600000 lines processed
3800000 lines processed
4000000 lines processed
4200000 lines processed
4400000 lines processed
4600000 lines processed
4800000 lines processed
5000000 lines processed
5200000 lines processed
5400000 lines processed
5600000 lines processed
5800000 lines processed
6000000 lines processed
6200000 lines processed
6400000 lines processed
6600000 lines processed
6800000 lines processed
7000000 lines processed
7200000 lines processed
7400000 lines processed
7600000 lines processed
7800000 lines processed
8000000 lines processed
8200000 lines processed
84

66200000 lines processed
66400000 lines processed
66600000 lines processed
66800000 lines processed
67000000 lines processed
67200000 lines processed
67400000 lines processed
67600000 lines processed
67800000 lines processed
68000000 lines processed
68200000 lines processed
68400000 lines processed
68600000 lines processed
68800000 lines processed
69000000 lines processed
69200000 lines processed
69400000 lines processed
69600000 lines processed
69800000 lines processed
70000000 lines processed
70200000 lines processed
70400000 lines processed
70600000 lines processed
70800000 lines processed
71000000 lines processed
71200000 lines processed
71400000 lines processed
71600000 lines processed
71800000 lines processed
72000000 lines processed
72200000 lines processed
72400000 lines processed
72600000 lines processed
72800000 lines processed
73000000 lines processed
73200000 lines processed
73400000 lines processed
73600000 lines processed
73800000 lines processed
74000000 lines processed


130600000 lines processed
130800000 lines processed
131000000 lines processed
131200000 lines processed
131400000 lines processed
131600000 lines processed
131800000 lines processed
132000000 lines processed
132200000 lines processed
132400000 lines processed
132600000 lines processed
132800000 lines processed
133000000 lines processed
133200000 lines processed
133400000 lines processed
133600000 lines processed
133800000 lines processed
134000000 lines processed
134200000 lines processed
134400000 lines processed
134600000 lines processed
134800000 lines processed
135000000 lines processed
135200000 lines processed
135400000 lines processed
135600000 lines processed
135800000 lines processed
136000000 lines processed
136200000 lines processed
136400000 lines processed
136600000 lines processed
136800000 lines processed
137000000 lines processed
137200000 lines processed
137400000 lines processed
137600000 lines processed
137800000 lines processed
138000000 lines processed
138200000 li

193800000 lines processed
194000000 lines processed
194200000 lines processed
194400000 lines processed
194600000 lines processed
194800000 lines processed
195000000 lines processed
195200000 lines processed
195400000 lines processed
195600000 lines processed
195800000 lines processed
196000000 lines processed
196200000 lines processed
196400000 lines processed
196600000 lines processed
196800000 lines processed
197000000 lines processed
197200000 lines processed
197400000 lines processed
197600000 lines processed
197800000 lines processed
198000000 lines processed
198200000 lines processed
198400000 lines processed
198600000 lines processed
198800000 lines processed
199000000 lines processed
199200000 lines processed
199400000 lines processed
199600000 lines processed
199800000 lines processed
200000000 lines processed
200200000 lines processed
200400000 lines processed
200600000 lines processed
200800000 lines processed
201000000 lines processed
201200000 lines processed
201400000 li

**Step 4:** Calculated the median of mean maximum trailing zeros.

In [48]:
k = 3
average_zeros = [np.mean(max_zero[i:i+k]) for i in range(0,40,k)]
np.median(average_zeros)

26.833333333333336

**Step 5:** Saved the file to CSV

In [24]:
averages = csv.writer(open("averages.csv",'w'), delimiter = ',', quoting = csv.QUOTE_ALL)
averages.writerow(average_zeros)

43

**Step 6:** Estimated number of unique quotes

In [49]:
unique = 2 ** np.median(average_zeros)
unique

119574401.82690892

### Question 4

Bloom Filter

**Step 1:** Imported all the necessary packages, and retrieved file list.

In [51]:
from bitarray import bitarray
import glob
import pyhash
import numpy as np
import sklearn
from sklearn.utils.murmurhash import murmurhash3_32

files = glob.glob("./Q2/*")
train_file = files[1]
test_file = files[0]

**Step 2:** Set the value of m as 350000, and built a bitarray of the same size, and set everything as false

In [115]:
m = 350000
bit = bitarray(m)
bit.setall(False)

**Step 3:** Built a function to count the number of lines in our files

In [116]:
def count(file):
    a = 0
    with open(file, 'r', encoding = 'utf8') as f:
        for line in f:
            a += 1
    return a

In [117]:
count_train = count(train_file)
print(count_train)

count_test = count(test_file)
print(count_test)

229943
2636692


**Step 4:** Calculated the optimal "k" hash functions on the basis of the length of bitarray m

In [118]:
k = (count_test/m)*np.log(2)
k

5.2217589308713235

**Step 5:** Found the value, and using the round value (k = 5). Implemented five hash functions (Murmur Hash, FNV Hash, XX Hash, Spooky Hash, and Farm Fingerprint Hash), calculated mod m of the value, and set the value of bitarray as true for the corresponding index value as the output of the computation. 

Trained the bitarray through the training set of only spams.

In [119]:
with open(train_file, 'r', encoding = 'utf8') as f:
    for line in f:
        murmur = murmurhash3_32(line, positive = True) % m
        fnv_32 = pyhash.fnv1_32()(line) % m
        xx_32 = pyhash.xx_32()(line) % m
        spooky_32 = pyhash.spooky_32()(line) % m
        farm_fingerprint32 = pyhash.farm_fingerprint_32()(line) % m
        bit[murmur] = True
        bit[fnv_32] = True
        bit[xx_32] = True
        bit[spooky_32] = True
        bit[farm_fingerprint32] = True

**Step 6:** Tried the same on the testing set, and depending on whether we get the higher count of True than False, the count of filter_in (unblocked) and filter_out (blocked).

In [120]:
filter_in = 0
filter_out = 0
with open(test_file, 'r', encoding = 'utf8') as f:
    for line in f:
        murmur = murmurhash3_32(line, positive = True) % m
        fnv_32 = pyhash.fnv1_32()(line) % m
        xx_32 = pyhash.xx_32()(line) % m
        spooky_32 = pyhash.spooky_32()(line) % m
        farm_fingerprint32 = pyhash.farm_fingerprint_32()(line) % m
        a = [bit[murmur], bit[fnv_32], bit[xx_32], bit[spooky_32], bit[farm_fingerprint32]]
        if a.count(False) < a.count(True):
            filter_out += 1
        else:
            filter_in += 1

**Step 7:** Calculated the false positive rate, which came out to be 0.04%

In [121]:
false_positive = filter_in / (filter_in + filter_out)
fpr = false_positive * 100
print("False Positive Rate:")
print(fpr)

False Positive Rate:
0.04460134137775667
